In [ ]:
!pip install deconvolution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from numba import njit
from deconvolution import Deconvolution
from PIL import Image
from skimage import io, color
from sklearn.cluster import KMeans

In [ ]:
def show_image(image, title='Image'):
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
def save_image(image, filename='image.png'):
    plt.imshow(image)
    plt.axis('off') 
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
def show_fullscreen_images(images, titles):
    for title in titles:
        cv2.namedWindow(title, cv2.WINDOW_NORMAL)
        cv2.setWindowProperty(title, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    screen_width = cv2.getWindowImageRect(titles[0])[2] 

    num_images = len(images)
    rows = int(np.ceil(num_images / 2))
    window_width = screen_width // 2
    window_height = screen_width // rows

    for i, (img, title) in enumerate(zip(images, titles)):
        window_width = int(window_width)
        window_height = int(window_height)

        resized_img = cv2.resize(img, (window_width, window_height))
        
        cv2.imshow(title, resized_img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def before_after(before_image, after_image, title_before="Before", title_after="After"):
    if before_image.dtype != np.uint8:
        before_image = (before_image * 255).astype(np.uint8)
    if after_image.dtype != np.uint8:
        after_image = (after_image * 255).astype(np.uint8)

    cv2.namedWindow(title_before, cv2.WINDOW_NORMAL)
    cv2.namedWindow(title_after, cv2.WINDOW_NORMAL)

    cv2.setWindowProperty(title_before, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.setWindowProperty(title_after, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    cv2.imshow(title_before, before_image)
    cv2.imshow(title_after, after_image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
image = io.imread('image003.jpg')
show_image(image)

## deconv

does not work

## scalar product

In [ ]:
rgb_values = np.array([
    [172, 16, 30],
    [83, 88, 130],
    [132, 195, 255]
])

def normalize_rgb(rgb):
    return list(map(lambda x: x / 255.0, rgb))

basis_vectors = list(map(normalize_rgb, rgb_values))

In [ ]:
@njit
def deconvolve_image_numba(img_np, basis_vectors):
    n_clusters = len(basis_vectors)
    output_images = [np.full_like(img_np, 255, dtype=np.float32) for _ in range(n_clusters)]

    white_threshold = 0.9  

    for i in range(img_np.shape[0]):
        for j in range(img_np.shape[1]):
            pixel = img_np[i, j]
            
            #if np.all(pixel > white_threshold):
            #    continue
            
            distances = np.zeros(n_clusters, dtype=np.float32)
            for k in range(n_clusters):
                distance = np.sqrt(np.sum((pixel - basis_vectors[k])**2))
                distances[k] = distance
            closest_index = np.argmin(distances)
            output_images[closest_index][i, j] = pixel

    return output_images

In [ ]:
def preprocess_and_postprocess(func):
    def wrapper(img, basis_vectors):
        img_np = np.array(img, dtype=np.float32) / 255.0
        basis_vectors_np = np.array(basis_vectors, dtype=np.float32)
        
        output_images = func(img_np, basis_vectors_np)
        
        output_images_uint8 = [
            (np.clip(img * 255, 0, 255)).astype(np.uint8) for img in output_images
        ]
        
        return output_images_uint8
    return wrapper


In [ ]:
@preprocess_and_postprocess
def deconvolve_image(img_np, basis_vectors):
    return deconvolve_image_numba(img_np, basis_vectors)

output_images = deconvolve_image(image, basis_vectors)

In [ ]:
for img in output_images:
    show_image(img)

### this is due to the blurriness of the image: noise on the seals, translucent signatures

## LAB color filtering

In [ ]:
lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
mask = cv2.inRange(lab_image, (0, 0, 0), (240, 255, 255)) 
cleaned_image = cv2.bitwise_and(image, image, mask=mask)

In [ ]:
before_after(image, cleaned_image)

### CLAHE 

In [ ]:
lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
l_channel, a, b = cv2.split(lab_image)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
cl = clahe.apply(l_channel)
lab_image = cv2.merge((cl, a, b))
enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

In [ ]:
before_after(image, enhanced_image)

### Gamma

In [ ]:
def adjust_gamma(image, gamma=1.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

enhanced_image = adjust_gamma(image, gamma=0.25)
before_after(image, enhanced_image)

### apply Gamma filter

In [ ]:
image = enhanced_image
show_image(image)

## remove noise

In [ ]:
def apply_median_blur(image, ksize=5):
    return cv2.medianBlur(image, ksize)

def apply_gaussian_blur(image, ksize=(5, 5)):
    return cv2.GaussianBlur(image, ksize, 0)

def apply_bilateral_filter(image, d=9, sigma_color=75, sigma_space=75):
    return cv2.bilateralFilter(image, d, sigma_color, sigma_space)

def apply_non_local_means_denoising(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)


image_median_blur = apply_median_blur(image, ksize=5)
image_gaussian_blur = apply_gaussian_blur(image, ksize=(5, 5))
image_bilateral = apply_bilateral_filter(image)
image_denoised = apply_non_local_means_denoising(image)

In [ ]:
# show_fullscreen_images([image, image_median_blur, image_gaussian_blur, image_bilateral, image_denoised], 
#                       ["Original", "Median Blur", "Gaussian Blur", "Bilateral Filter", "Non-Local Means"])

## apply gaussian blur

In [ ]:
image = image_gaussian_blur
show_image(image)

In [ ]:
cv2.imwrite("enhanced_image.png", enhanced_image)

## try clustering again

In [ ]:
rgb_values = np.array([
    [206, 26, 2],
    [23, 3, 3],
    [8, 5, 193]
])

def normalize_rgb(rgb):
    return list(map(lambda x: x / 255.0, rgb))

basis_vectors = list(map(normalize_rgb, rgb_values))

In [ ]:
output_images = deconvolve_image(image, basis_vectors)
for img in output_images:
    show_image(img)

In [ ]:
for idx, img in enumerate(output_images):
    cv2.imwrite(f"result_{idx}.jpg", img)